# Plotting Results of Crypto Strategy


In [ ]:
# from config import general as config
# from config import crypto
import pandas as pd
import numpy as np
# matplotlib.use('Agg')
%matplotlib inline
import pyfolio
import matplotlib.pyplot as plt
from pyfolio import timeseries
import os
from datetime import datetime
import empyrical as ep
# from finrl.plot import backtest_stats, get_daily_return # , backtest_plot, get_daily_return, get_baseline

colors = ["#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
          "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000"]

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def get_daily_return_from_intraday(df, value_col_name="account_value"):
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    df["return"] = df[value_col_name].pct_change(1) + 1
    groupper = df.groupby(pd.Grouper(key='date', freq='1d'))
    return groupper['return'].prod() -1

def backtest_stats_intraday(daily_returns):
    # pyfolio perf_stats() doesnt handle 365-day data for a year
    # perf_stats_all = timeseries.perf_stats(returns=daily_returns, positions=None, transactions=None, turnover_denom="AGB")   
    stats = {}
    stats['min_date'] = str(daily_returns.index[0].strftime("%Y-%m-%d %H:%M"))
    stats['max_date'] = str(daily_returns.index[-1].strftime("%Y-%m-%d %H:%M"))
    stats['cum_ret'] = (daily_returns + 1).prod() - 1
    stats['ann_ret'] = ep.annual_return(daily_returns, annualization=365)    
    stats['ann_vol'] = ep.annual_volatility(daily_returns, annualization=365)
    stats['sharpe'] = ep.sharpe_ratio(daily_returns, risk_free=0, annualization=365)
    stats['sortino'] = ep.sortino_ratio(daily_returns, annualization=365)    
    stats['max_dd'] = ep.max_drawdown(daily_returns)    
    return stats
    
def append_stats(stats, filename):
    # print(f"append to {RESULTS_DIR}/strategy_performance.csv")
    x = pd.DataFrame(stats, index=[0])    
    x.to_csv(filename, float_format='{:.6f}'.format, mode='a', index=False, header=False)
    
def get_time_resolution(t1, t2):
    delta = (t2-t1)
    # print(delta)
    if delta.days == 1:
        return '1d'
    if delta.seconds == 3600:
        return '1h'
    if delta.seconds == 43200:
        return '12h'
    if delta.seconds == 21600:
        return '6h'
    if delta.seconds == 1800:
        return '30min'
    if delta.seconds == 300:
        return '5min'
    return f"{delta.seconds}s"

def print_stats(stats):    
    print(f"Strategy:     {stats['strategy']}")
    print(f"Model:        {stats['model']}")
    print(f"Date Range:   {stats['min_date']} - {stats['max_date']}")
    print(f"Resolution:   {stats['res']}")
    print(f"-----------------------------------------------------")    
    print(f"Cumulative Return :   {stats['cum_ret']:7.4f}")
    print(f"Annual Return     :   {stats['ann_ret']:7.4f}")
    print(f"Annual Volatility :   {stats['ann_vol']:7.4f}")
    print(f"Sharpe ratio      :   {stats['sharpe']:7.4f}")
    print(f"Sortino ratio     :   {stats['sortino']:7.4f}")
    print(f"Max Drawdown      :   {stats['max_dd']:7.4f}")

def print_baseline(name, stats):
    print("============== Baseline Benchmark ===========")
    print(f"Model:        {name}")
    print(f"Date Range:   {stats['min_date']} - {stats['max_date']}")
    print(f"-----------------------------------------------------")    
    print(f"Cumulative Return :   {stats['cum_ret']:7.4f}")
    print(f"Annual Return     :   {stats['ann_ret']:7.4f}")
    print(f"Annual Volatility :   {stats['ann_vol']:7.4f}")
    print(f"Sharpe ratio      :   {stats['sharpe']:7.4f}")
    print(f"Sortino ratio     :   {stats['sortino']:7.4f}")
    print(f"Max Drawdown      :   {stats['max_dd']:7.4f}")
    
def generate_backtest_results_crypto(acc_val, strat_name, model_name, run_name, printstats=True):
    
    acc_val['date'] = pd.to_datetime(acc_val['date'])
    time_res = get_time_resolution(acc_val.iloc[0]['date'], acc_val.iloc[1]['date'])
    
    print("============== Backtest Results ===========")
    daily_returns = get_daily_return_from_intraday(acc_val, value_col_name="account_value")
    stats = backtest_stats_intraday(daily_returns)
    
    parsed_stats = {}
    parsed_stats['strategy'] = strat_name
    parsed_stats['model'] = model_name
    parsed_stats['run_name'] = run_name
    parsed_stats['min_date'] = stats['min_date']
    parsed_stats['max_date'] = stats['max_date']
    parsed_stats['res'] = time_res
    parsed_stats['cum_ret'] = stats['cum_ret']
    parsed_stats['ann_ret'] = stats['ann_ret']
    parsed_stats['ann_vol'] = stats['ann_vol']
    parsed_stats['sharpe'] = stats['sharpe']
    parsed_stats['sortino'] = stats['sortino']
    parsed_stats['max_dd'] = stats['max_dd']
    parsed_stats['created'] = datetime.now().strftime("%Y-%m-%d %H:%M")

    if printstats:
        print_stats(parsed_stats)
    # append_stats(parsed_stats)
    # perf_stats_df = pd.DataFrame(perf_stats_intraday)
    # perf_stats_df.to_csv(f"{RESULTS_FILE_PREFIX}_perf_stats.csv")
    return stats, daily_returns, parsed_stats

def get_baseline(names, acc_val, return_idx=0):
    single_baselines = []
    start = acc_val.loc[0, 'date']
    end = acc_val.loc[len(acc_val) - 1, 'date']
    selected_baseline = None

    for i, baseline_name in enumerate(names):
        baseline_returns = get_baseline_returns_crypto(baseline_name, start, end)
        if i == return_idx:
            selected_baseline = baseline_returns
        x = pd.DataFrame(backtest_stats_intraday(baseline_returns), index=[baseline_name])
        single_baselines.append(x)

    baseline_stats = pd.concat(single_baselines)
    return baseline_stats, selected_baseline

def get_baseline_returns_crypto(baseline_strategy='MPT', start='2022-06-01 00:00:00', end='2022-11-01 23:59:59'):
    df = pd.read_csv(f"datasets/thesis/benchmark/1d_benchmark_returns.csv", index_col=0)
    df.index = pd.to_datetime(df.index)
    baseline = df[baseline_strategy]
    baseline = baseline -1
    baseline = baseline[(baseline.index >= start ) & (baseline.index <= end)]
    return baseline

def backtest_plot_crypto(strategy_returns, baseline_returns, baseline_name='BTCUSDT'):
    print(f"==============Compare to {baseline_name}===========")
    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns=strategy_returns, benchmark_rets=baseline_returns, set_context=False)
        
def get_asset_ratios(acc_val):
    names = ['AVAXUSDT', 'AXSUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'LINKUSDT', 'LTCUSDT', 'SHIBUSDT', 'TLMUSDT', 'UNIUSDT']
    ratio_df = acc_val[["date", "cash"]].copy()
    for n in names:
        ratio_df[n] = acc_val[f"{n}_price"] * acc_val[f"{n}_amount"]
        # ratio_df[n] = ratio_df[n].where(ratio_df[n] < 0, 0)
    # ratio_df['all_assets_without_cash'] = ratio_df[[f"{x}" for x in names]].sum(axis=1)
    ratio_df[names] = ratio_df[names].clip(lower=0)
    for n in names:
        ratio_df[n] = ratio_df[n] / acc_val[f"account_value"]
        
    ratio_df["cash"] = ratio_df["cash"] / acc_val[f"account_value"]
    ratio_df["all_assets"] = ratio_df[names].sum(axis=1)
    return ratio_df

In [ ]:
ROOT_DIR = '.'
MODEL_NAME = "TD3"
STRAT_NAME = "cs_eval"
RUN_NAME = "EVAL_TD3_V203_12201307_12M"
FILE_PREFIX = "all"
# RUN_NAME = "A2C_V221_12130953_5M"
RESULTS_DIR=f"../results"

RESULTS_FILE_PREFIX = f"{ROOT_DIR}/{RESULTS_DIR}/{STRAT_NAME}/{MODEL_NAME}/{RUN_NAME}/{FILE_PREFIX}"
RESULTS_DIR = f"{ROOT_DIR}/{RESULTS_DIR}/{STRAT_NAME}/"

In [ ]:
filename = f"{RESULTS_FILE_PREFIX}_state.csv"
results_account_value = pd.read_csv(filename)

In [ ]:
ratios = get_asset_ratios(results_account_value)

In [ ]:
stats, strat_returns = generate_backtest_results_crypto(results_account_value, STRAT_NAME, MODEL_NAME, RUN_NAME)

In [ ]:
stats

In [ ]:
list(stats.keys())
list(stats.values())

In [ ]:
x = pd.DataFrame(list(stats.values())).T
x.columns = list(stats.keys())
x

In [ ]:
baseline_index = 2
baseline_stats, baseline_returns = get_baseline(['MPT', 'BAH', 'CRP'], results_account_value, baseline_index)
# baseline_stats.to_csv(f"results/baseline.csv")
# pd.concat([pd.DataFrame(stats, index=['strategy']), baseline_stats]).T
print_baseline(baseline_returns.name, baseline_stats.iloc[baseline_index])

In [ ]:
names = ['AVAXUSDT', 'AXSUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'LINKUSDT', 'LTCUSDT', 'SHIBUSDT', 'TLMUSDT', 'UNIUSDT', 'cash']
fig = plt.figure(figsize=(12,4), tight_layout=True)
plt.stackplot(ratios.date, *[ratios[col] for col in names], labels=(names), colors=colors, baseline='zero')
plt.legend(ncols=1, bbox_to_anchor=(1.2, 1.0))
plt.savefig(f"{RESULTS_DIR}/weights.png")

In [ ]:
pyfolio.plotting.show_perf_stats(strat_returns, baseline_returns)

In [ ]:
drawdown_df = pyfolio.timeseries.gen_drawdown_table(strat_returns, top=5)

In [ ]:
pyfolio.plotting.show_worst_drawdown_periods(strat_returns)

In [ ]:
pyfolio.tears.create_interesting_times_tear_sheet(strat_returns, benchmark_rets=baseline_returns, set_context=False)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,5))
pyfolio.plotting.plot_drawdown_periods(strat_returns, top=5, ax=ax)
plt.savefig(f"{RESULTS_DIR}/stats_drawdown_periods.png")

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,5))
ax = pyfolio.plotting.plot_drawdown_underwater(returns=strat_returns, ax=ax)
plt.savefig(f"{RESULTS_DIR}/stats_underwater.png")

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
ax = pyfolio.plotting.plot_monthly_returns_heatmap(strat_returns, ax=ax)

In [ ]:
backtest_plot_crypto(strat_returns, baseline_returns)

In [ ]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [ ]:
runs = get_immediate_subdirectories('../results/ce')

model = "ensemble"
STRAT_NAME = "ce"
FILE_PREFIX = "all"
RESULTS_DIR = f"../results/{STRAT_NAME}"

for run in runs:
    print(f"{model} - {run}")
    RESULTS_FILE_PREFIX = f"{RESULTS_DIR}/{run}/{FILE_PREFIX}"  
    filename = f"{RESULTS_FILE_PREFIX}_state.csv"
    # print(filename)
    results_account_value = pd.read_csv(filename)
    stats, strat_returns, parsed_stats = generate_backtest_results_crypto(results_account_value, STRAT_NAME, model, run, printstats=False)
    append_stats(parsed_stats, f'{RESULTS_DIR}/strategy_performance.csv')

In [ ]:
stats, strat_returns, parsed_stats = generate_backtest_results_crypto(results_account_value, STRAT_NAME, model, run)
append_stats(parsed_stats, f'{RESULTS_DIR}/strategy_performance.csv')